In [ ]:
from src.pipelines.pipeline_highres_sdxl import StableDiffusionXLHighResPipeline
import torch

from src.linfusion import LinFusion
from src.tools import seed_everything

In [ ]:
model_ckpt = "stabilityai/stable-diffusion-xl-base-1.0"
device = torch.device('cuda')
pipe = StableDiffusionXLHighResPipeline.from_pretrained(
    model_ckpt, torch_dtype=torch.float16, variant='fp16'
).to(device)

In [ ]:
linfusion = LinFusion.construct_for(pipe)

In [ ]:
prompt = "An astronaut floating in space. Beautiful view of the stars and the universe in the background."
generator = torch.manual_seed(42)
pipe.enable_vae_tiling()
images = pipe(prompt,
              height=1024, width=2048, device=device,
              num_inference_steps=50, guidance_scale=7.5,
              cosine_scale_1=3, cosine_scale_2=1, cosine_scale_3=1, gaussian_sigma=0.8,
              show_image=True, generator=generator, upscale_strength=0.32)